In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [4]:
# 폴리곤을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [5]:
# 라인스트링을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [6]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [9]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in crp_df.iterrows():
    popup_text = f"{row['properties.구역명']}"
    folium.GeoJson(
        row['centroid_polygon_geometry'].__geo_interface__,
        style_function=lambda feature, color='blue': {'fillColor': 'blue', 'color': 'black', 'weight': 0.1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 격자(매핑용)

In [11]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df

,type,properties.gid,geometry.type,geometry.coordinates,geometry
0,Feature,라바122456,Polygon,"[[[127.63641867895493, 36.608165744671886], [1...",POLYGON ((127.63641867895493 36.60816574467188...
1,Feature,다바841455,Polygon,"[[[127.32221067151384, 36.60720970472786], [12...",POLYGON ((127.32221067151384 36.60720970472786...
2,Feature,라바189471,Polygon,"[[[127.71137365974974, 36.62157886605511], [12...",POLYGON ((127.71137365974974 36.62157886605511...
3,Feature,다바852482,Polygon,"[[[127.33445848271631, 36.631567839444976], [1...",POLYGON ((127.33445848271631 36.63156783944497...
4,Feature,다바959488,Polygon,"[[[127.45413724007568, 36.63708300728262], [12...",POLYGON ((127.45413724007568 36.63708300728262...
...,...,...,...,...,...
95251,Feature,다바868478,Polygon,"[[[127.35236168440159, 36.62798537617458], [12...",POLYGON ((127.35236168440159 36.62798537617458...
95252,Feature,다바974324,Polygon,"[[[127.47097166337947, 36.48924077421588], [12...",POLYGON ((127.47097166337947 36.48924077421588...
95253,Feature,라바151492,Polygon,"[[[127.66891675313607, 36.640578073348856], [1...",POLYGON ((127.66891675313607 36.64057807334885...
95254,Feature,다바841537,Polygon,"[[[127.3220406982908, 36.681132105366395], [12...",POLYGON ((127.3220406982908 36.681132105366395...


In [12]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.01)

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_grid.iterrows():
    popup_text = f"{row['properties.gid']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'fillColor': 'black', 'color': 'black', 'weight': 0.1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 거주인구 격자에 매핑

In [ ]:
pd.read_csv('SBJ_2309_001/1.청주시_거주인구.csv')

In [ ]:
pd.read_csv('SBJ_2309_001/2.청주시_성연령별_유동인구.csv')

### 청주시 폐공장현황

In [ ]:
factory_df = pd.read_csv('SBJ_2309_001/9.청주시_폐공장현황.csv')
factory_df

In [11]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
factory_df = geo_transform(factory_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_factory = factory_df[factory_df['geometry'].within(polygon)]

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_factory.iterrows():
    marker = folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 문화재 현황 (폴리곤 영역확장 변경으로 교통분석 후 문화재 필터링 해야할 듯)

https://m.blog.naver.com/zukyun59/222997801975 - 역사문화전경 보존지역 관련 글

In [214]:
culture_assets_df = pd.read_csv('SBJ_2309_001/5.청주시_유적지_문화재현황.csv')

In [210]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
culture_assets_df = geo_transform(culture_assets_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# 폴리곤을 확장 (예: 0.1 도 또는 10 킬로미터 확장)
expanded_polygon = polygon.buffer(0.015)  # 0.015은 확장할 크기. EPSG 4326 좌표계에서는 위도 및 경도가 0.015도씩 확장됨.(약 1.7km 나중에 변경 될 수 있음)

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_culture = culture_assets_df[culture_assets_df['geometry'].within(expanded_polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [211]:
filtered_points_culture

,gbn,clt_nm,addr,lon,lat,geometry
0,국보,청주 용두사지 철당간,충청북도 청주시 상당구 남문로2가 48-19,127.489758,36.633095,POINT (127.48976 36.63309)
1,보물,청주 용화사 석조불상군,충청북도 청주시 서원구 사직동 216-1,127.482003,36.641779,POINT (127.48200 36.64178)
2,충북도 유형문화재,청주 충청도병마절도사영문,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
3,충북도 유형문화재,청주 탑동 오층석탑,충청북도 청주시 상당구 탑동 251,127.497987,36.629600,POINT (127.49799 36.62960)
4,충북도 유형문화재,청주향교,충청북도 청주시 상당구 대성동 68,127.498060,36.637203,POINT (127.49806 36.63720)
5,충북도 유형문화재,청주 청녕각,충청북도 청주시 상당구 북문로1가 171-3,127.488321,36.634704,POINT (127.48832 36.63470)
6,충북도 유형문화재,청주 망선루,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
7,충북도 유형문화재,청주 탑동 양관,충청북도 청주시 상당구 탑동 185-1,127.496569,36.627226,POINT (127.49657 36.62723)
8,충북도 유형문화재,청주 조헌 전장기적비,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
9,충북도 유형문화재,청주 성공회 성당,충청북도 청주시 상당구 수동 202-1,127.493948,36.638636,POINT (127.49395 36.63864)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_culture.iterrows():
    popup_text = f"{row['gbn']} - {row['clt_nm']}건"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m).add_child(folium.Popup(popup_text, max_width=100))

# 지도 저장
m

### 청주시 시장 현황 (폴리곤 영역확장 없이 범위 내 시장들만 필터링)

In [217]:
market_df = pd.read_csv('SBJ_2309_001/6.청주시_시장현황.csv')
market_df

,market_nm,addr,area,lon,lat
0,육거리종합시장,상당구 청남로2197번길 42(석교동),86273,127.488184,36.627938
1,서문시장,상당구 남사로89번길 57(서문동),10285,127.485365,36.634894
2,중앙시장,상당구 중앙로 26(북문로2가),7305,127.489192,36.639385
3,사직시장,서원구 사직대로265번길 7(사직동),9200,127.476868,36.636143
4,북부시장,청원구 향군로31번길 19(우암동),22133,127.486221,36.647973


In [218]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
market_df = geo_transform(market_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_market = market_df[market_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [219]:
filtered_points_market

,market_nm,addr,area,lon,lat,geometry
0,육거리종합시장,상당구 청남로2197번길 42(석교동),86273,127.488184,36.627938,POINT (127.48818 36.62794)
1,서문시장,상당구 남사로89번길 57(서문동),10285,127.485365,36.634894,POINT (127.48536 36.63489)
2,중앙시장,상당구 중앙로 26(북문로2가),7305,127.489192,36.639385,POINT (127.48919 36.63938)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_market.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 주차장 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [ ]:
parking_df = pd.read_csv('SBJ_2309_001/7.청주시_주차장현황.csv')
parking_df

In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
parking_df = geo_transform(parking_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_parking = parking_df[parking_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
filtered_points_parking # 노상 : 도로 위 한켠에 주차를 하고 일정 비용을 받음(정부 또는 지자체 관할), 노외 : 공용주차장처럼 별도 대지에 설치되어 제공되는 주차장

In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_parking.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 공원 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [ ]:
park_df = pd.read_csv('SBJ_2309_001/8.청주시_공원현황.csv')
park_df

In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
park_df = geo_transform(park_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_park = park_df[park_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
filtered_points_park

In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_park.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 학교 현황

In [ ]:
school_df = pd.read_csv('SBJ_2309_001/24.청주시_학교현황.csv')
school_df

In [128]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
school_df = geo_transform(school_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_school = school_df[school_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_school

In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_school.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 어린이집 유치원 현황

In [ ]:
kinder_df = pd.read_csv('SBJ_2309_001/28.청주시_어린이집_유치원현황.csv')
kinder_df

In [39]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
kinder_df = geo_transform(kinder_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_kinder = kinder_df[kinder_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_kinder

In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_kinder.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 가로수 현황 (공공데이터포털 - 산림청_도시숲가로수관리 가로수 현황 data https://www.data.go.kr/data/15120900/fileData.do)

In [ ]:
'''산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'''
# street_tree_df = pd.read_csv('SBJ_2309_001/산림청_도시숲가로수관리 가로수 현황_20221231.csv', encoding='cp949')
# street_tree_df = street_tree_df[['시군구명', '도로구간명', '수종명', '지역X좌표', '지역Y좌표']]
# street_tree_df = street_tree_df[street_tree_df['시군구명'].str.contains('청주')].reset_index(drop=True)

# # EPSG 5186에서 EPSG 4326으로 좌표 변환을 수행하는 함수 생성
# def transform_coordinates(row):
#     in_proj = pyproj.Proj(init='epsg:5186')
#     out_proj = pyproj.Proj(init='epsg:4326')
#     lon, lat = pyproj.transform(in_proj, out_proj, row['지역X좌표'], row['지역Y좌표'])
#     return pd.Series({'lat': lat, 'lon': lon})

# # '지역X좌표'와 '지역Y좌표'를 EPSG 4326으로 변환하고 'lat' 및 'lon' 열로 이름 변경
# street_tree_df[['lat', 'lon']] = street_tree_df.apply(transform_coordinates, axis=1)
# # '지역X좌표'와 '지역Y좌표' 열을 삭제 (선택 사항)
# street_tree_df.drop(['지역X좌표', '지역Y좌표'], axis=1, inplace=True)

# 

In [5]:
street_tree_df = pd.read_csv('SBJ_2309_001/29.청주시_가로수현황.csv', encoding='UTF8')
street_tree_df

,시군구명,도로구간명,수종명,lat,lon
0,충청북도 청주시 청원구,오창대로,벚나무,36.708975,127.481089
1,충청북도 청주시 청원구,오창대로,벚나무,36.708883,127.480707
2,충청북도 청주시 청원구,오창대로,벚나무,36.708996,127.481787
3,충청북도 청주시 청원구,오창대로,벚나무,36.708625,127.480314
4,충청북도 청주시 청원구,오창대로,벚나무,36.708993,127.481158
...,...,...,...,...,...
24481,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660599,127.507634
24482,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660668,127.507612
24483,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660737,127.507589
24484,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660249,127.507748


In [6]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
street_tree_df = geo_transform(street_tree_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_tree = street_tree_df[street_tree_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_tree

In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_tree.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 상세 도로망 시각화

In [9]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/13.청주시_상세도로망.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.02) # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadsystem = roadsystem_df[roadsystem_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadsystem

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadsystem.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m)
    
m

### 청주시 불법주정차

In [179]:
illegal_parking_df = pd.read_csv('SBJ_2309_001/14.청주시_불법주정차단속통계.csv')
illegal_parking_df = geo_transform(illegal_parking_df)
illegal_parking_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,cd_type,cd_loc,cd_dong,cd_cnt,lon,lat,geometry
0,고정형,상당로 남궁요양병원사거리부근1,문화동,13,127.490864,36.632164,POINT (127.49086 36.63216)
1,고정형,사직대로 대현지하상가나도약국부근,북문로1가,15,127.487698,36.636797,POINT (127.48770 36.63680)
2,고정형,사직대로 지하상가성안길입구부근,북문로2가,17,127.488849,36.636973,POINT (127.48885 36.63697)
3,고정형,상당로 도청사거리부근,북문로1가,20,127.490597,36.634411,POINT (127.49060 36.63441)
4,고정형,무심동로 육거리주차장부근,석교동,20,127.487675,36.627044,POINT (127.48768 36.62704)
5,고정형,청소년광장사거리부근,북문로2가,272,127.488643,36.639136,POINT (127.48864 36.63914)
6,고정형,홈플러스후문사거리부근,서문동,374,127.488256,36.635447,POINT (127.48826 36.63545)
7,고정형,상당로81번길 산업은행청주점부근,북문로1가,133,127.488747,36.635504,POINT (127.48875 36.63550)
8,고정형,사직대로350번길 청주상호저축은행부근,서문동,129,127.486397,36.634354,POINT (127.48640 36.63435)
9,고정형,교서로 청주공고부근,영동,52,127.486449,36.637435,POINT (127.48645 36.63744)


In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

for _, row in illegal_parking_df.iterrows() :
    popup_text = f"{row['cd_loc']}({row['cd_dong']} - {row['cd_cnt']}건)"
    folium.Circle(location=(row['lat'], row['lon']), radius=30, color='#FF580B',
            fill='#FF580B').add_to(m).add_child(folium.Popup(popup_text, max_width=200))
    
m

### 청주시 112신고 통계 (격자매핑 형식)

In [181]:
police_report_df = pd.read_csv('SBJ_2309_001/20.청주시_112신고통계.csv')
# 5개년 신고건수 총합
police_report_df['repo_total'] = police_report_df['repo_2018'] + police_report_df['repo_2019'] + police_report_df['repo_2020'] + police_report_df['repo_2021'] + police_report_df['repo_2022']
police_report_df['repo_total'] = police_report_df['repo_total'].astype(int)
police_report_df = police_report_df[['gid', 'repo_total']]

#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_police_report = police_report_df[police_report_df['gid'].isin(grid_id)]
filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)
filtered_police_report.reset_index(drop=True, inplace=True)
filtered_police_report

C:\Users\user\AppData\Local\Temp\ipykernel_2952\1370399623.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
C:\Users\user\AppData\Local\Temp\ipykernel_2952\1370399623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)


,gid,repo_total,geometry
0,다바989482,1519,POLYGON ((127.48769621544852 36.63168220843648...
1,다바988483,1510,"POLYGON ((127.4865775332584 36.63258358652596,..."
2,다바992477,1406,POLYGON ((127.49105231410148 36.62717501134686...
3,다바991478,953,POLYGON ((127.48993373614363 36.62807642164767...
4,다바990486,887,POLYGON ((127.4888142201003 36.635288313298695...
...,...,...,...
268,다바994489,2,"POLYGON ((127.49328829744827 36.637993143918, ..."
269,다바996484,2,POLYGON ((127.49552579225157 36.63348575733207...
270,다바984494,2,POLYGON ((127.48210108412849 36.64249947656094...
271,다바995485,1,POLYGON ((127.49440717516781 36.63438720867091...


#### 경찰신고수는 평균에 대해 얼마나 높은가를 구해서 시각화 해야할듯...?

KeplerGl로 시각화 해보기

In [184]:
len(filtered_police_report)

273

In [186]:
# 데이터프레임에서 데이터 추출
lst = filtered_police_report['repo_total']

# 조건에 따라 각 막대의 색상을 설정
colors = ['green' if x > 600 else 'yellow' if x > 300 else 'red' for x in lst]

# 히스토그램을 생성
fig = go.Figure(go.Histogram(x=lst, marker_color=colors, nbinsx=1000))

# 레이아웃 설정
fig.update_xaxes(title_text='report_count')
fig.update_yaxes(title_text='count')
fig.update_layout(title_text='report_count_histogram')

# 플롯리 그래프 보이기
fig.show()

In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 신고건수에 따라 색상을 지정하는 함수
def color_producer(repo_total):
    if repo_total < 300:
        return 'green'
    elif 300 <= repo_total < 800:
        return 'yellow'
    else:
        return 'red'
        
for _, row in filtered_police_report.iterrows() :
    popup_text = f"격자:{row['gid']}(신고건수 - {row['repo_total']}건)"
    color = color_producer(row['repo_total'])
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
m

### 청주시 카드매출 (격자매핑)

KB국민카드 데이터를 활용하기 보다는 총 매출 추정금액을 활용하는 것이 더 효율적일 것으로 예상됨

In [56]:
cardsales_df = pd.read_csv('SBJ_2309_001/19.청주시_카드매출_격자매핑.csv')
cardsales_df.dropna(inplace=True)
cardsales_df.reset_index(drop=True, inplace=True)
cardsales_df['est_sales'] = (cardsales_df['est_sales'] * 1000).astype(int)
cardsales_df
#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_cardsales = cardsales_df[cardsales_df['gid'].isin(grid_id)]
# 'est_sales' 열의 값이 0인 행을 'card_sales' 값으로 채우기
filtered_cardsales['est_sales'] = filtered_cardsales.apply(lambda row: row['card_sales'] if row['est_sales'] == 0 else row['est_sales'], axis=1)
filtered_cardsales

C:\Users\user\AppData\Local\Temp\ipykernel_2952\4040690819.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cardsales['est_sales'] = filtered_cardsales.apply(lambda row: row['card_sales'] if row['est_sales'] == 0 else row['est_sales'], axis=1)


,stdr_ym,kbc_bzc_nm_1,kbc_bzc_nm_2,kbc_bzc_nm_3,card_sales,est_sales,gid
0,202305,서비스,(정기)납부/대여서비스,미디어/음향,989000,18995000,다바989487
1,202305,서비스,(정기)납부/대여서비스,도서/음반대여,54000,844000,다바989487
2,202305,서비스,의료서비스,한의원,5019200,83333000,다바992490
3,202305,소매업,의류,여성의류,1600000,25979000,다바990484
4,202305,소매업,패션잡화,구두제화,103500,3315000,다바989483
...,...,...,...,...,...,...,...
13006,202306,소매업,의류,일반의류(기타),1657400,1657400,다바990481
13007,202305,음식,커피/음료,커피전문점,98200,98200,다바987489
13008,202306,소매업,패션잡화,액세서리,109010,109010,다바988486
13009,202305,음식,기타외국식,쌀국수전문점,8000,8000,다바991482


In [57]:
# 격자 id : 격자 폴리곤 매핑 딕셔너리
map_dic = {}
for _, row in filtered_grid.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

In [58]:
geometry_lst = []
for _, row in filtered_cardsales.iterrows():
    geometry_lst.append(map_dic[row['gid']])
filtered_cardsales['geometry'] = geometry_lst

C:\Users\user\AppData\Local\Temp\ipykernel_2952\1955721771.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cardsales['geometry'] = geometry_lst


In [ ]:
filtered_cardsales_month

In [151]:
# 'gid' 값을 기준으로 그룹화하고 'est_sales' 합계 리스트 미리 생성, 상위 33%, 중위 33%, 하위 33%의 임계값 탐색
est_lst = []

for gid in filtered_cardsales_month['gid'].unique():
    subset = filtered_cardsales_month[filtered_cardsales_month['gid'] == gid].copy()
    subset['kbc_total'] = subset.iloc[:, 1] + ' - ' +  subset.iloc[:, 3]
    est_sales_sum = subset['est_sales'].sum()
    est_lst.append(est_sales_sum)

# 데이터를 정렬
sorted_sales = sorted(est_lst)

# 상위 33%, 중위 33%, 하위 33%의 임계값을 찾음
num_data_points = len(est_lst)

# 하위 33%와 상위 33%에 해당하는 임계값 계산
lower_threshold = sorted_sales[int(len(sorted_sales) * 0.33)]
upper_threshold = sorted_sales[int(len(sorted_sales) * 0.67)]

#  매출 액수에 따라 색상을 지정하는 함수
def color_picker(est_sales_sum):
    if est_sales_sum < lower_threshold:
        return 'green'
    elif lower_threshold <= est_sales_sum < upper_threshold:
        return 'yellow'
    else:
        return 'red'

In [ ]:
filtered_cardsales_month = filtered_cardsales[filtered_cardsales['stdr_ym'] == 202302]

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     top: 50px; right: 50px; width: 500px; height: 125px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "><br> &nbsp; 총매출 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 총매출 상위 33% ({1:,}원 이상)<br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 총매출 중위 33% ({0:,}원 이상 & {1:,}원 미만)<br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 총매출 하위 33% ({0:,}원 이하)<br>
     </div>
     """.format(lower_threshold, upper_threshold)

# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=13)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 'gid' 값을 기준으로 그룹화
for gid in filtered_cardsales_month['gid'].unique():
    subset = filtered_cardsales_month[filtered_cardsales_month['gid'] == gid].copy()
    subset['kbc_total'] = subset.iloc[:, 1] + ' - ' +  subset.iloc[:, 3]
    est_sales_sum = subset['est_sales'].sum()
    
    # 매출 데이터를 3자리마다 쉼표로 구분하고 "원"을 추가
    subset['카테고리'] = subset['kbc_total'].copy()
    subset['총매출'] = subset['est_sales'].apply(lambda x: f'{x:,}원')
        
    # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
    subset_popup = subset[['카테고리', '총매출']].style.hide(axis='index').set_properties(**{'border': '1px solid black'}).to_html()
    popup_text = f"격자 ID - {gid}<br>총매출 - {est_sales_sum:,}원<br><br><div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
    color = color_picker(est_sales_sum)
    folium.GeoJson(
        filtered_cardsales_month[filtered_cardsales_month['gid'] == gid]['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=1200))

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))

m

### 청주시 도로명주소(건물 + 도로)

In [53]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/15.청주시_도로명주소(건물).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_building_df = pd.json_normalize(geojson_data['features'])
roadname_building_df = roadname_building_df[['properties.BD_MGT_SN', 'properties.GRO_FLO_CO', 'properties.UND_FLO_CO', 'geometry.coordinates']]
roadname_building_df['properties.GRO_FLO_CO'] = roadname_building_df['properties.GRO_FLO_CO'].astype(int)
roadname_building_df['properties.UND_FLO_CO'] = roadname_building_df['properties.UND_FLO_CO'].astype(int)
roadname_building_df['geometry'] = roadname_building_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [ ]:
roadname_building_df

In [54]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
roadname_building_df = gpd.GeoDataFrame(roadname_building_df, geometry='geometry')

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_building = roadname_building_df[roadname_building_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadname_building

,properties.BD_MGT_SN,properties.GRO_FLO_CO,properties.UND_FLO_CO,geometry.coordinates,geometry
0,4311110100100600000061755,5,0,"[[[127.48694447846057, 36.6400016341025], [127...","POLYGON ((127.48694 36.64000, 127.48694 36.639..."
1,4311110100100550002032236,1,0,"[[[127.48632680823786, 36.64052917039099], [12...","POLYGON ((127.48633 36.64053, 127.48633 36.640..."
2,4311110100100560003032233,2,0,"[[[127.48655786043167, 36.640535159309565], [1...","POLYGON ((127.48656 36.64054, 127.48646 36.640..."
3,4311110100100560003071386,1,0,"[[[127.48653912832565, 36.640421876670125], [1...","POLYGON ((127.48654 36.64042, 127.48646 36.640..."
4,4311110100100560002032235,3,0,"[[[127.48657086929198, 36.64041777756533], [12...","POLYGON ((127.48657 36.64042, 127.48656 36.640..."
...,...,...,...,...,...
3184,4311110800100300004000001,4,0,"[[[127.49253748307129, 36.63019961400344], [12...","POLYGON ((127.49254 36.63020, 127.49279 36.630..."
3185,4311110100100040003046395,1,0,"[[[127.48581567013639, 36.644730907282785], [1...","POLYGON ((127.48582 36.64473, 127.48588 36.644..."
3186,4311110800100590001049620,5,0,"[[[127.49155463874942, 36.63035531760507], [12...","POLYGON ((127.49155 36.63036, 127.49155 36.630..."
3187,4311110900101650001000001,3,0,"[[[127.48496189891529, 36.63447898239693], [12...","POLYGON ((127.48496 36.63448, 127.48514 36.634..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_building.iterrows():
    popup_text = f"지상:{row['properties.GRO_FLO_CO']}층, 지하:{row['properties.UND_FLO_CO']}층)"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

https://thlee33.medium.com/kepler-gl%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B1%B4%EB%AC%BC-3d-%EC%8B%9C%EA%B0%81%ED%99%94-330400887fe3

In [86]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/16.청주시_도로명주소(도로).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')

In [ ]:
roadname_road_df

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_road = roadname_road_df[roadname_road_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadname_road

In [ ]:
filtered_roadname_road

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_road.iterrows():
    popup_text = f"도로명:{row['properties.RN']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 인도(보도) 시각화

In [115]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')

In [ ]:
sidewalk_df

In [117]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_sidewalk

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706050A003189376b14e5964828,미분류,무,인도,LineString,"[[127.49062338110333, 36.6417421675925], [127....","LINESTRING (127.49062 36.64174, 127.49064 36.6..."
1,Feature,100036706050A003101d5f9fbadf84488,미분류,무,인도,LineString,"[[127.48804192410691, 36.63751484423243], [127...","LINESTRING (127.48804 36.63751, 127.48806 36.6..."
2,Feature,100036706050A00314c2a46c79bfa4ef8,미분류,무,인도,LineString,"[[127.4908535706583, 36.6425890088251], [127.4...","LINESTRING (127.49085 36.64259, 127.49079 36.6..."
3,Feature,100036706050A003158cc0af359c945e6,미분류,무,인도,LineString,"[[127.48891283162526, 36.6390439338725], [127....","LINESTRING (127.48891 36.63904, 127.48889 36.6..."
4,Feature,100036706050A00315cf42551cbd149da,미분류,무,인도,LineString,"[[127.48642173123054, 36.63556500043981], [127...","LINESTRING (127.48642 36.63557, 127.48645 36.6..."
...,...,...,...,...,...,...,...,...
116,Feature,100036706050A00316998a65ea4ad440f,아스콘,유,자전거도로,LineString,"[[127.49112468574835, 36.631374516942394], [12...","LINESTRING (127.49112 36.63137, 127.49110 36.6..."
117,Feature,100036706050A0031db4c79d55cb84d39,블록,유,자전거도로,LineString,"[[127.49075989252123, 36.63456720470323], [127...","LINESTRING (127.49076 36.63457, 127.49073 36.6..."
118,Feature,100036706050A0031dc05c4cbc88a4fe9,아스콘,유,자전거도로,LineString,"[[127.491361592062, 36.62943927534968], [127.4...","LINESTRING (127.49136 36.62944, 127.49132 36.6..."
119,Feature,100036706050A0031881c48839a5441fb,블록,유,자전거도로,LineString,"[[127.49060087641541, 36.63567678716627], [127...","LINESTRING (127.49060 36.63568, 127.49049 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_sidewalk.iterrows():
    popup_text = f"도로명:{row['properties.KIND']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 상권정보 시각화

In [ ]:
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
trading_area_df

['음식', '소매', '생활서비스', '학문/교육', '부동산', '관광/여가/오락', '숙박', '스포츠'],

In [207]:
a = filtered_trading_area[filtered_trading_area['com_lc_nm'] == '소매']
a['com_mc_nm'].unique()

array(['건강/미용식품', '의복의류', '가정/주방/인테리어', '사진/광학/정밀기기소매', '화장품소매',
       '음/식료품소매', '선물/팬시/기념품', '시계/귀금속소매', '철물/난방/건설자재소매', '유아용품',
       '종합소매점', '책/서적/도서', '가방/신발/액세서리', '사무/문구/컴퓨터', '자동차/자동차용품',
       '애견/애완/동물', '가전제품소매', '가구소매', '의약/의료품소매', '중고품소매/교환', '취미/오락관련소매',
       '운동/경기용품소매', '예술품/골동품/수석/분재', '페인트/유리제품소매', '기타판매업', '종교용품판매'],
      dtype=object)

In [205]:
filtered_trading_area['com_lc_nm'].value_counts()

com_lc_nm
소매          1629
음식           869
생활서비스        470
학문/교육         84
관광/여가/오락      49
숙박            36
부동산           34
스포츠            7
Name: count, dtype: int64

In [208]:
a['com_mc_nm'].value_counts()

com_mc_nm
의복의류             568
종합소매점            187
가방/신발/액세서리       162
음/식료품소매          118
화장품소매            101
가정/주방/인테리어        83
사진/광학/정밀기기소매      70
시계/귀금속소매          52
선물/팬시/기념품         41
건강/미용식품           38
가구소매              31
사무/문구/컴퓨터         28
의약/의료품소매          21
기타판매업             21
운동/경기용품소매         20
철물/난방/건설자재소매      19
책/서적/도서           16
가전제품소매            16
유아용품               7
자동차/자동차용품          6
애견/애완/동물           6
취미/오락관련소매          6
중고품소매/교환           5
예술품/골동품/수석/분재      4
종교용품판매             2
페인트/유리제품소매         1
Name: count, dtype: int64

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_trading_area

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_trading_area.iterrows():
    popup_text = f"{row['com_lc_nm']}-{row['com_mc_nm']}-{row['com_sc_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 주민등록인구 현황

In [231]:
pop_df = pd.read_csv('SBJ_2309_001/22.청주시_주민등록인구현황.csv')
pop_df

,adm_cd,adm_nm,year,m_pop,fm_pop
0,4311131000,충청북도 청주시 상당구 낭성면,201701,1232,1140
1,4311132000,충청북도 청주시 상당구 미원면,201701,2757,2636
2,4311133000,충청북도 청주시 상당구 가덕면,201701,2259,2066
3,4311134000,충청북도 청주시 상당구 남일면,201701,3901,3746
4,4311135000,충청북도 청주시 상당구 문의면,201701,2383,2165
...,...,...,...,...,...
3091,4311451000,충청북도 청주시 청원구 우암동,202212,6209,6073
3092,4311452000,충청북도 청주시 청원구 내덕1동,202212,4197,4124
3093,4311453000,충청북도 청주시 청원구 내덕2동,202212,6823,6453
3094,4311454000,충청북도 청주시 청원구 율량.사천동,202212,24328,24478


In [233]:
pop_df['adm_nm'].unique()

array(['충청북도 청주시 상당구 낭성면', '충청북도 청주시 상당구 미원면', '충청북도 청주시 상당구 가덕면',
       '충청북도 청주시 상당구 남일면', '충청북도 청주시 상당구 문의면', '충청북도 청주시 상당구 중앙동',
       '충청북도 청주시 상당구 성안동', '충청북도 청주시 상당구 탑대성동', '충청북도 청주시 상당구 영운동',
       '충청북도 청주시 상당구 금천동', '충청북도 청주시 상당구 용담.명암.산성동', '충청북도 청주시 상당구 용암1동',
       '충청북도 청주시 상당구 용암2동', '충청북도 청주시 서원구 남이면', '충청북도 청주시 서원구 현도면',
       '충청북도 청주시 서원구 사직1동', '충청북도 청주시 서원구 사직2동', '충청북도 청주시 서원구 사창동',
       '충청북도 청주시 서원구 모충동', '충청북도 청주시 서원구 산남동', '충청북도 청주시 서원구 분평동',
       '충청북도 청주시 서원구 수곡1동', '충청북도 청주시 서원구 수곡2동', '충청북도 청주시 서원구 성화.개신.죽림동',
       '충청북도 청주시 흥덕구 오송읍', '충청북도 청주시 흥덕구 강내면', '충청북도 청주시 흥덕구 옥산면',
       '충청북도 청주시 흥덕구 운천.신봉동', '충청북도 청주시 흥덕구 복대1동', '충청북도 청주시 흥덕구 복대2동',
       '충청북도 청주시 흥덕구 가경동', '충청북도 청주시 흥덕구 봉명1동', '충청북도 청주시 흥덕구 봉명2.송정동',
       '충청북도 청주시 흥덕구 강서제1동', '충청북도 청주시 흥덕구 강서제2동', '충청북도 청주시 청원구 내수읍',
       '충청북도 청주시 청원구 오창읍', '충청북도 청주시 청원구 북이면', '충청북도 청주시 청원구 우암동',
       '충청북도 청주시 청원구 내덕1동', '충청북도 청주시 청원구 내덕2동', '충청북도 청주시 청원구 율량.사천동',
       '충청북도 청주시 청원구 오근장

## turtle로 그림그리는 방안 생각해보기?